In [1]:
pip install langchain-community duckduckgo-search

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install -U ddgs

Note: you may need to restart the kernel to use updated packages.


In [3]:
import datetime
from typing import TypedDict, List, Dict, Optional
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, END

# --- 1. CONFIGURATION ---
# Using Qwen 2.5 (0.5B)
llm = ChatOllama(model="qwen2.5:0.5b", temperature=0)

# --- 2. DEFINE STATE ---
class PlannerState(TypedDict):
    subjects: List[str]
    total_hours: float
    difficulty_map: str       # Stores the "Hard/Easy" analysis
    schedule_text: str        # The final timetable
    resources: List[str]      # The YouTube links
    quality_score: int        # 0-10 rating
    feedback: str             # Auditor notes

# --- 3. DEFINE NODES ---

def difficulty_analyzer_node(state: PlannerState):
    """
    Agent 1: The Strategist.
    Decides which subject is Hard or Easy.
    """
    print(f"🧠 Strategist: Analyzing difficulty for {state['subjects']}...")
    
    prompt = ChatPromptTemplate.from_template(
        """Analyze these subjects: {subjects}.
        Assign 'Hard', 'Medium', or 'Easy' to each.
        Return ONLY a simple list like: Math: Hard, Art: Easy.
        """
    )
    chain = prompt | llm
    response = chain.invoke({"subjects": state['subjects']})
    
    return {"difficulty_map": response.content}

def scheduler_node(state: PlannerState):
    """
    Agent 2: The Architect.
    Creates the table.
    """
    now = datetime.datetime.now().strftime("%I:%M %p")
    print(f"📅 Architect: Building schedule starting at {now}...")
    
    # We explicitly ask for a Markdown Table here
    prompt = ChatPromptTemplate.from_template(
        """You are a strict scheduler.
        Current Time: {current_time}
        Total Hours: {hours}
        Difficulty: {diff_map}
        Subjects: {subjects}
        
        Task: Create a study schedule in a Markdown Table format.
        Columns: [Time Slot] | [Subject] | [Activity/Topic]
        
        Rules:
        1. Start exactly at {current_time}.
        2. Include 10-minute breaks between subjects.
        3. Do not chat. Output ONLY the table.
        """
    )
    
    chain = prompt | llm
    response = chain.invoke({
        "current_time": now,
        "hours": state['total_hours'],
        "subjects": state['subjects'],
        "diff_map": state['difficulty_map']
    })
    
    return {"schedule_text": response.content}

def resource_generator_node(state: PlannerState):
    """
    Agent 3: The Librarian (Fixed).
    Generates YouTube links instead of searching text snippets.
    """
    print("🔗 Librarian: Generating YouTube links...")
    subjects = state["subjects"]
    links = []
    
    for sub in subjects:
        # Create a clickable YouTube Search URL
        query = sub.replace(" ", "+")
        url = f"https://www.youtube.com/results?search_query={query}+tutorial"
        links.append(f"📺 **{sub}**: {url}")
            
    return {"resources": links}

def auditor_node(state: PlannerState):
    """
    Agent 4: The Critic.
    """
    print("⚖️ Auditor: Checking table formatting...")
    plan = state["schedule_text"]
    
    # Check if it actually looks like a table
    if "|" in plan and "-" in plan:
        score = 9
    else:
        score = 4
        
    print(f"   Score: {score}/10")
    
    if score < 6:
        return {"quality_score": score, "feedback": "Output was not a table."}
    return {"quality_score": score, "feedback": "Approved"}

# --- 4. ROUTER LOGIC ---

def route_auditor(state: PlannerState):
    if state["quality_score"] < 6:
        print("   ❌ Retrying schedule...")
        return "scheduler"
    else:
        print("   ✅ Schedule Approved!")
        return "resource_generator"

# --- 5. BUILD GRAPH ---

workflow = StateGraph(PlannerState)

workflow.add_node("strategist", difficulty_analyzer_node)
workflow.add_node("scheduler", scheduler_node)
workflow.add_node("auditor", auditor_node)
workflow.add_node("resource_generator", resource_generator_node)

workflow.set_entry_point("strategist")
workflow.add_edge("strategist", "scheduler")
workflow.add_edge("scheduler", "auditor")

workflow.add_conditional_edges(
    "auditor",
    route_auditor,
    {
        "scheduler": "scheduler",
        "resource_generator": "resource_generator"
    }
)

workflow.add_edge("resource_generator", END)

app = workflow.compile()

# --- 6. RUN IT ---

inputs = {
    "subjects": ["Machine Learning", "Linear Algebra"],
    "total_hours": 3
}

final_state = app.invoke(inputs, config={"recursion_limit": 10})

print("\n" + "="*40)
print("       🎓 YOUR STUDY PLAN 🎓")
print("="*40 + "\n")

print(f"📊 Analysis:\n{final_state['difficulty_map']}\n")
print(f"🗓️ TIMETABLE:\n{final_state['schedule_text']}\n")
print("-" * 30)
print("🌍 RESOURCES:")
for res in final_state['resources']:
    print(res)

🧠 Strategist: Analyzing difficulty for ['Machine Learning', 'Linear Algebra']...
📅 Architect: Building schedule starting at 11:01 PM...
⚖️ Auditor: Checking table formatting...
   Score: 9/10
   ✅ Schedule Approved!
🔗 Librarian: Generating YouTube links...

       🎓 YOUR STUDY PLAN 🎓

📊 Analysis:
Math: Hard, Art: Medium.

🗓️ TIMETABLE:
| Time Slot | Subject   | Activity/Topic |
|-----------|----------|-----------------|
| 11:01    | Machine Learning | Linear Algebra     |
| 11:02    | Math       | Calculus          |
| 11:03    | Physics    | Quantum Mechanics |
| 11:04    | History    | World History      |
| 11:05    | Art        | Painting         |
| 11:06    | Music      | Classical Music  |
| 11:07    | Science   | Biology          |
| 11:08    | English   | Literature       |
| 11:09    | History  | World History     |
| 11:10    | Art        | Painting         |
| 11:11    | Music      | Classical Music  |
| 11:12    | Science   | Biology          |
| 11:13    | English   | Lit